In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
# pip install -q -U tensorflow-addons

In [3]:
# import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D,LocallyConnected1D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization,Add,concatenate
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [4]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [5]:
N_FEATURES = 12

In [6]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'Class_label']


In [7]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [8]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [9]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [10]:
 pd.options.display.float_format = "{:,.5f}".format

In [11]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [12]:
# import os
path='/media/naveen/nav/mat_codes/nina_DB2_codes/prep_nl/Total_process_TT/'

In [13]:
#  for i in range(num_res_net_blocks_64):
#     x = res_net_block_64(x, 64, 3)

In [14]:
# file_path_train=path+'Train_data'+'.'+'csv'
file_path_train=path+'nina_pro_nl_DB2_Train'+'.'+'csv'
print(file_path_train)

/media/naveen/nav/mat_codes/nina_DB2_codes/prep_nl/Total_process_TT/nina_pro_nl_DB2_Train.csv


In [15]:
file_path_test=path+'nina_pro_nl_DB2_Test'+'.'+'csv'
# file_path_test=path+'Test_data'+'.'+'csv'
print(file_path_test)

/media/naveen/nav/mat_codes/nina_DB2_codes/prep_nl/Total_process_TT/nina_pro_nl_DB2_Test.csv


In [16]:
df_Train=read_data_Train(file_path_train)
show_basic_dataframe_info(df_Train)
df_Train.head(5)

Number of columns in the dataframe: 13
Number of rows in the dataframe: 78400000



,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,Class_label
0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000
1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000
2,0.00000,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00001,0.00000,0.00000,0.00000,0.00000,1.00000
3,0.00000,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00001,0.00000,0.00000,0.00000,0.00000,1.00000
4,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000


In [17]:
df_Test=read_data_Test(file_path_test)
show_basic_dataframe_info(df_Test)
df_Test.head(5)

Number of columns in the dataframe: 13
Number of rows in the dataframe: 39200000



,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,Class_label
0,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00001,0.00001,0.00000,0.00000,0.00000,0.00002,1.00000
1,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00001,0.00001,0.00000,0.00000,0.00000,0.00002,1.00000
2,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00001,0.00000,0.00000,0.00000,0.00002,1.00000
3,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00001,0.00000,0.00000,0.00000,0.00002,1.00000
4,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00001,0.00000,0.00000,0.00000,0.00002,1.00000


In [18]:
pd.options.display.float_format = "{:,.5f}".format
df_Train.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,Class_label
0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000
1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000
2,0.00000,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00001,0.00000,0.00000,0.00000,0.00000,1.00000
3,0.00000,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00001,0.00000,0.00000,0.00000,0.00000,1.00000
4,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000


In [19]:
scaler = preprocessing.StandardScaler().fit(df_Train.iloc[:,0:N_FEATURES])

In [20]:
df_Train.iloc[:,0:N_FEATURES]=scaler.transform(df_Train.iloc[:,0:N_FEATURES])

In [21]:
pd.options.display.float_format = "{:,.5f}".format
df_Train.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,Class_label
0,-0.25708,-0.17073,-0.39718,-0.28539,-0.18949,-0.14848,-0.20964,-0.50601,-0.33990,-0.21971,-0.28669,-0.10038,1.00000
1,-0.27204,-0.15374,-0.40034,-0.29309,-0.17334,-0.15399,-0.20801,-0.47291,-0.33520,-0.22135,-0.26047,-0.06350,1.00000
2,-0.27538,-0.13066,-0.37366,-0.30159,-0.16145,-0.16008,-0.20686,-0.39567,-0.32823,-0.22318,-0.23210,-0.02916,1.00000
3,-0.26442,-0.13164,-0.33945,-0.30109,-0.12619,-0.16477,-0.20695,-0.39122,-0.32851,-0.22455,-0.23511,-0.05554,1.00000
4,-0.25744,-0.14498,-0.31252,-0.29566,-0.09751,-0.16776,-0.20773,-0.42531,-0.33256,-0.22553,-0.25903,-0.11322,1.00000


In [22]:
pd.options.display.float_format = "{:,.5f}".format
df_Test.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,Class_label
0,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00001,0.00001,0.00000,0.00000,0.00000,0.00002,1.00000
1,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00001,0.00001,0.00000,0.00000,0.00000,0.00002,1.00000
2,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00001,0.00000,0.00000,0.00000,0.00002,1.00000
3,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00001,0.00000,0.00000,0.00000,0.00002,1.00000
4,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00001,0.00000,0.00000,0.00000,0.00002,1.00000


In [23]:
df_Test.iloc[:,0:N_FEATURES]=scaler.transform(df_Test.iloc[:,0:N_FEATURES])

In [24]:
pd.options.display.float_format = "{:,.5f}".format
df_Test.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,Class_label
0,-0.18005,-0.19843,-0.30437,-0.30440,-0.38860,-0.15674,-0.17638,-0.25397,-0.27909,-0.23257,-0.30719,1.02360,1.00000
1,-0.18744,-0.19819,-0.30099,-0.31208,-0.33263,-0.16695,-0.18634,-0.27912,-0.27832,-0.22866,-0.30485,1.12191,1.00000
2,-0.20211,-0.19969,-0.29713,-0.32090,-0.26583,-0.16685,-0.19750,-0.32002,-0.27797,-0.22470,-0.29710,1.28405,1.00000
3,-0.22303,-0.20434,-0.29280,-0.31134,-0.23188,-0.15577,-0.20644,-0.36249,-0.27834,-0.22244,-0.27375,1.44824,1.00000
4,-0.24734,-0.21051,-0.28937,-0.30535,-0.21907,-0.13608,-0.21283,-0.40769,-0.27918,-0.22166,-0.25072,1.65328,1.00000


In [25]:
LABEL = 'ActivityEncoded'
# Transform the labels from String to Integer via LabelEncoder
le = preprocessing.LabelEncoder()
# Add a new column to the existing DataFrame with the encoded values
df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
# df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())

In [26]:
# The number of steps within one time segment
TIME_PERIODS = 400
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 400

In [27]:
x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
print('x_train shape: ', x_train.shape)
# print(x_train)
print(x_train.shape[0], 'training samples')
print('y_train shape: ', y_train.shape)
# Set input & output dimensions
num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
print('num_time_periods',num_time_periods)
print('num_sensors',num_sensors)
num_classes = le.classes_.size
print('class_list',list(le.classes_))
# input_shape = (num_time_periods,num_sensors)
# print(input_shape)
input_shape = (num_time_periods,num_sensors)
#x_train = x_train.reshape(x_train.shape[0], input_shape)
print('x_train shape:', x_train[0].shape)
print('input_shape:', input_shape)
x_train = x_train.astype('float32')
# x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
# y_train = y_train.astype('float32')
# print(y_train)
y_train_hot = np_utils.to_categorical(y_train, num_classes)
print(y_train_hot)
# y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
print('New y_train shape: ', y_train_hot.shape)

x_train shape:  (195999, 400, 12)
195999 training samples
y_train shape:  (195999,)
num_time_periods 400
num_sensors 12
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0]
x_train shape: (400, 12)
input_shape: (400, 12)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
New y_train shape:  (195999, 49)


In [28]:
x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
print('x_test shape: ', x_test.shape)
# print(x_train)
print(x_test.shape[0], 'testing samples')
print('y_test shape: ', y_test.shape)
# Set input_shape / reshape for Keras
#x_test = x_test.reshape(x_test.shape[0], input_shape)
x_test = x_test.astype('float32')
y_test = y_test.astype('float32')
y_test_hot = np_utils.to_categorical(y_test, num_classes)

x_test shape:  (97999, 400, 12)
97999 testing samples
y_test shape:  (97999,)


In [29]:
# n_steps, n_length = 20, 25
# n_steps, n_length= 10, 50
# n_steps, n_length= 16, 32
n_length =  400
n_depth=12
n_channel=12
x_train = x_train.reshape(x_train.shape[0], n_length,n_depth)
print('x_train shape: ', x_train.shape)
# x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
# print('x_valid shape: ', x_valid.shape)
x_test = x_test.reshape(x_test.shape[0],  n_length,n_depth)
print('x_test shape: ', x_test.shape)
n_outputs = y_train_hot.shape[1]
print('n_outputs',n_outputs)

x_train shape:  (195999, 400, 12)
x_test shape:  (97999, 400, 12)
n_outputs 49


In [30]:
def conv_block(input_data,n_channel):
    inputs=input_data
    print(inputs.shape)
    l_y=[]
    for i in range(0,n_channel):
        ip=inputs[:,:,i]
        print('ip_shape',ip.shape)
#         ip1=tf.convert_to_tensor(ip, dtype=tf.float32)
        ip=tf.reshape(ip,(tf.shape(ip)[0],400,1))
        print('ip_shape',ip.shape)
        x = Conv1D(64, kernel_size=3,kernel_initializer="he_normal",kernel_regularizer=l1(1e-04), \
               padding='same',input_shape=ip.shape)(ip)
        print('conv1_o',x.shape)
        x = BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x = Activation('relu')(x)
        x= Conv1D(filters=64, kernel_size=3,padding="same", kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1)(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        y= Dropout(0.5)(x)
        l_y.append(y)
    y_stack=tf.stack(l_y, axis=2)   
    return y_stack

In [31]:
def generate_model():
    inputs = Input(shape=(n_length,n_depth),name="main")
    p=conv_block(inputs,12)
    print(p.shape)
    print(type(p))
#     print(y.shape)
    y=Flatten()(p)
    print(y.shape)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(128, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    outputs=Dense(n_outputs, activation='softmax')(y)
    print(outputs.shape)
#     wei_mscnn_model = Model(inputs,sub_inputs], outputs)
    print('inputs_shape',inputs.shape)
    final_model = Model(inputs,outputs,name="wei_mscnn_model")
    return final_model        

In [32]:
if __name__ == "__main__":
    model = generate_model()

(None, 400, 12)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
(None, 400, 12, 64)
<class 'tensorflow.python.framework.ops.Tensor'>
(None, 307200)
(None, 49)
inputs_shape (None, 400, 12)


In [43]:
verbose, epochs, batch_size = 0, 26, 32

In [44]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 12.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [45]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [46]:
adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, amsgrad=False)
# sgd=tfa.optimizers.SGDW(weight_decay=0.0001,learning_rate=0.1, momentum=0.9, nesterov=False, name='SGDW')
sgd=tf.optimizers.SGD(learning_rate=1e-3, momentum=0.9, nesterov=False, name='SGD')
checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB2_20X10/checkpoint.hdf5'
model.load_weights(checkpoint_filepath) 
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)

Model: "wei_mscnn_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main (InputLayer)               [(None, 400, 12)]    0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 400)]        0           main[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_strided_slice_2 (Te [(None, 400)]        0           main[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_strided_slice_4 (Te [(None, 400)]        0           main[0][0]                       
____________________________________________________________________________________

In [47]:
csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB2_20X10/CNN_nina_20X10.csv', append=True, separator=';')
history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)

Epoch 1/26
6125/6125 [==============================] - ETA: 0s - loss: 2.8594 - accuracy: 0.2296
Epoch 00001: val_accuracy improved from -inf to 0.21093, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB2_20X10/checkpoint.hdf5
6125/6125 [==============================] - 2112s 345ms/step - loss: 2.8594 - accuracy: 0.2296 - val_loss: 3.1230 - val_accuracy: 0.2109 - lr: 0.0010
Epoch 2/26
6125/6125 [==============================] - ETA: 0s - loss: 2.8491 - accuracy: 0.2310
Epoch 00002: val_accuracy improved from 0.21093 to 0.28063, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB2_20X10/checkpoint.hdf5
6125/6125 [==============================] - 1987s 324ms/step - loss: 2.8491 - accuracy: 0.2310 - val_loss: 2.6814 - val_accuracy: 0.2806 - lr: 0.0010
Epoch 3/26
6125/6125 [==============================] - ETA: 0s - loss: 2.8418 - accuracy: 0.2328
Epoch 00003: val_accuracy improved from 0.28063 to 0.28725, saving model to

6125/6125 [==============================] - ETA: 0s - loss: 2.6290 - accuracy: 0.2702
Epoch 00026: val_accuracy did not improve from 0.31413
6125/6125 [==============================] - 2030s 331ms/step - loss: 2.6290 - accuracy: 0.2702 - val_loss: 2.6141 - val_accuracy: 0.2972 - lr: 1.0000e-05


In [48]:
best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
print('epoch_number',best_index+1)
print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])

epoch_number 12
train accuracy and validation accuracy 0.2523431181907654 0.314125657081604


In [49]:
model.load_weights(checkpoint_filepath) 
_, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
print('test_accuracy',testaccuracy)
test_acc.append(testaccuracy)
print(test_acc)

3063/3063 [==============================] - 216s 70ms/step - loss: 2.5365 - accuracy: 0.3141
test_accuracy 0.314125657081604
[0.314125657081604]


In [50]:
statistics.mean(test_acc)

0.314125657081604